## Credit card balance time series feature extraction
Train GRU network on credit card balance time series data. Save prediction to be used as features in final training.

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
import gc

import os
print(os.listdir("../input"))
    
gc.enable()

['download_command.txt', 'application_test.csv', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


Scale data for NN training.

In [2]:
def scale_data(df_):
    df = df_.copy(deep=True)
    for f_ in df_.columns:
        if (df[f_].max()- df[f_].min() <=10):
            df[f_] = df[f_] - df[f_].min()
            continue
        df[f_] = df[f_] - df[f_].median()
        scale = (df[f_].quantile(0.99)-df[f_].quantile(0.01))
        if scale==0:
            scale = df[f_].max() - df[f_].min()
        df[f_] = df[f_]/scale
        if df[f_].max()>10:
            rescale = df[f_]>df[f_].quantile(0.99)
            quantile99 = df[f_].quantile(0.99)
            quantile100 = df[f_].max()
            df[f_].loc[rescale] = quantile99 + (df[f_].loc[rescale] - quantile99) * (10-quantile99)/(quantile100-quantile99)
        if df[f_].min()<-10:
            rescale = df[f_]<df[f_].quantile(0.01)
            quantile1 = df[f_].quantile(0.01)
            quantile0 = df[f_].min()
            df[f_].loc[rescale] = quantile1 + (df[f_].loc[rescale] - quantile1) * (-10-quantile1)/(quantile0-quantile1)
        df[f_] = df[f_] - df[f_].min()
    return df

Read credit card balance data and create features.

In [3]:
ccbl = pd.read_csv('../input/credit_card_balance.csv')
    
ccbl = pd.concat([ccbl, pd.get_dummies(ccbl['NAME_CONTRACT_STATUS'], prefix='NAME_CONTRACT_STATUS')], axis=1)
del ccbl['NAME_CONTRACT_STATUS']

sum_feats = [f_ for f_ in ccbl.columns.values if ((f_.find('SK_ID_CURR')<0) & (f_.find('MONTHS_BALANCE')<0) & (f_.find('SK_ID_PREV')<0))]
print('sum_feats',sum_feats)
sum_ccbl_mon = ccbl.groupby(['SK_ID_CURR','MONTHS_BALANCE'])[sum_feats].sum()
sum_ccbl_mon['CNR_ACCOUNT_W_MONTH'] = ccbl.groupby(['SK_ID_CURR','MONTHS_BALANCE'])['SK_ID_PREV'].count()
ccbl = sum_ccbl_mon.reset_index()

#compute ratio after summing up account
ccbl['AMT_BALANCE_CREDIT_RATIO'] = (ccbl['AMT_BALANCE']/(ccbl['AMT_CREDIT_LIMIT_ACTUAL']+0.001)).clip(-100,100)
ccbl['AMT_CREDIT_USE_RATIO'] = (ccbl['AMT_DRAWINGS_CURRENT']/(ccbl['AMT_CREDIT_LIMIT_ACTUAL']+0.001)).clip(-100,100)
ccbl['AMT_DRAWING_ATM_RATIO'] = ccbl['AMT_DRAWINGS_ATM_CURRENT']/(ccbl['AMT_DRAWINGS_CURRENT']+0.001)
ccbl['AMT_DRAWINGS_OTHER_RATIO'] = ccbl['AMT_DRAWINGS_OTHER_CURRENT']/(ccbl['AMT_DRAWINGS_CURRENT']+0.001)
ccbl['AMT_DRAWINGS_POS_RATIO'] = ccbl['AMT_DRAWINGS_POS_CURRENT']/(ccbl['AMT_DRAWINGS_CURRENT']+0.001)
ccbl['AMT_PAY_USE_RATIO'] = ((ccbl['AMT_PAYMENT_TOTAL_CURRENT']+0.001)/(ccbl['AMT_DRAWINGS_CURRENT']+0.001)).clip(-100,100)
ccbl['AMT_BALANCE_RECIVABLE_RATIO'] = ccbl['AMT_BALANCE']/(ccbl['AMT_TOTAL_RECEIVABLE']+0.001)
ccbl['AMT_DRAWING_BALANCE_RATIO'] = ccbl['AMT_DRAWINGS_CURRENT']/(ccbl['AMT_BALANCE']+0.001)
ccbl['AMT_RECEIVABLE_PRINCIPAL_DIFF'] = ccbl['AMT_TOTAL_RECEIVABLE']-ccbl['AMT_RECEIVABLE_PRINCIPAL']
ccbl['AMT_PAY_INST_DIFF'] = ccbl['AMT_PAYMENT_CURRENT'] - ccbl['AMT_INST_MIN_REGULARITY']

rejected_features = ['AMT_RECIVABLE','AMT_RECEIVABLE_PRINCIPAL','AMT_DRAWINGS_ATM_CURRENT',
                     'AMT_DRAWINGS_OTHER_CURRENT','AMT_DRAWINGS_POS_CURRENT']
for f_ in rejected_features:
    del ccbl[f_]
    
ccbl.iloc[:,3:] = scale_data(ccbl.iloc[:,3:])

del sum_ccbl_mon
gc.collect()
ccbl.head()

sum_feats ['AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD', 'SK_DPD_DEF', 'NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Approved', 'NAME_CONTRACT_STATUS_Completed', 'NAME_CONTRACT_STATUS_Demand', 'NAME_CONTRACT_STATUS_Refused', 'NAME_CONTRACT_STATUS_Sent proposal', 'NAME_CONTRACT_STATUS_Signed']


/var/folders/cm/wy6qs4m10756wz1k16wsjx000000gn/T/ipykernel_85776/4069996604.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f_].loc[rescale] = quantile99 + (df[f_].loc[rescale] - quantile99) * (10-quantile99)/(quantile100-quantile99)
/var/folders/cm/wy6qs4m10756wz1k16wsjx000000gn/T/ipykernel_85776/4069996604.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f_].loc[rescale] = quantile99 + (df[f_].loc[rescale] - quantile99) * (10-quantile99)/(quantile100-quantile99)
/var/folders/cm/wy6qs4m10756wz1k16wsjx000000gn/T/ipykernel_85776/4069996604.py:16: SettingWithCopyWarning: 
A value is trying to be set on a c

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,...,AMT_BALANCE_CREDIT_RATIO,AMT_CREDIT_USE_RATIO,AMT_DRAWING_ATM_RATIO,AMT_DRAWINGS_OTHER_RATIO,AMT_DRAWINGS_POS_RATIO,AMT_PAY_USE_RATIO,AMT_BALANCE_RECIVABLE_RATIO,AMT_DRAWING_BALANCE_RATIO,AMT_RECEIVABLE_PRINCIPAL_DIFF,AMT_PAY_INST_DIFF
0,100006,-6,0.0,0.352941,0.039439,0.0,0.0,0.0,0.904595,0.0,...,10.0,0.025281,10.0,0.0,0.0,1.01,0.217997,10.0,10.0,1.550076
1,100006,-5,0.0,0.352941,0.039439,0.0,0.0,0.0,0.904595,0.0,...,10.0,0.025281,10.0,0.0,0.0,1.01,0.217997,10.0,10.0,1.550076
2,100006,-4,0.0,0.352941,0.039439,0.0,0.0,0.0,0.904595,0.0,...,10.0,0.025281,10.0,0.0,0.0,1.01,0.217997,10.0,10.0,1.550076
3,100006,-3,0.0,0.352941,0.039439,0.0,0.0,0.0,0.904595,0.0,...,10.0,0.025281,10.0,0.0,0.0,1.01,0.217997,10.0,10.0,1.550076
4,100006,-2,0.0,0.352941,0.039439,0.0,0.0,0.0,0.904595,0.0,...,10.0,0.025281,10.0,0.0,0.0,1.01,0.217997,10.0,10.0,1.550076


Read target from main table.

In [4]:
data_app = pd.read_csv('../input/application_train.csv',usecols=['SK_ID_CURR','TARGET'])
data_test = pd.read_csv('../input/application_test.csv',usecols=['SK_ID_CURR'])
data_app.shape, data_test.shape

((307511, 2), (48744, 1))

In [5]:
trn_id = data_app['SK_ID_CURR'].loc[data_app.SK_ID_CURR.isin(ccbl.SK_ID_CURR)]
test_id = data_test['SK_ID_CURR'].loc[data_test['SK_ID_CURR'].isin(ccbl.SK_ID_CURR)]
trn_id.shape, test_id.shape

((86905,), (16653,))

Split train and test set. Group by ID and month to create time series.

In [6]:
ccbl_trn = ccbl.loc[ccbl.SK_ID_CURR.isin(trn_id)]
ccbl_test = ccbl.loc[ccbl.SK_ID_CURR.isin(test_id)]
feats = ccbl.columns.values[2:]
ccbl_trn = ccbl_trn.groupby(['SK_ID_CURR','MONTHS_BALANCE'])[feats].sum() 
ccbl_test = ccbl_test.groupby(['SK_ID_CURR','MONTHS_BALANCE'])[feats].sum() 
#tmp = tmp.reset_index() 
#table = pd.pivot_table(tmp, index='SK_ID_CURR', columns='MONTHS_BALANCE', values=feats, fill_value=0) 
#table.head(10)
ccbl_test.head()

AMT_BALANCE  AMT_CREDIT_LIMIT_ACTUAL  \
SK_ID_CURR MONTHS_BALANCE                                         
100013     -96                     0.0                 0.205882   
           -95                     0.0                 0.205882   
           -94                     0.0                 0.205882   
           -93                     0.0                 0.205882   
           -92                     0.0                 0.205882   

                           AMT_DRAWINGS_CURRENT  AMT_INST_MIN_REGULARITY  \
SK_ID_CURR MONTHS_BALANCE                                                  
100013     -96                         0.039439                      0.0   
           -95                         0.039439                      0.0   
           -94                         0.039439                      0.0   
           -93                         0.039439                      0.0   
           -92                         0.039439                      0.0   

                           AMT_PAYMENT_CURRENT  AMT_PAYMENT_TOTAL_CURRENT  \
SK_ID_CURR MONTHS_BALANCE                                                   
100013     -96                             0.0                        0.0   
           -95                             0.0                        0.0   
           -94                             0.0                        0.0   
           -93                             0.0                        0.0   
           -92                             0.0                        0.0   

                           AMT_TOTAL_RECEIVABLE  CNT_DRAWINGS_ATM_CURRENT  \
SK_ID_CURR MONTHS_BALANCE                                                   
100013     -96                         0.904595                       0.0   
           -95                         0.904595                       0.0   
           -94                         0.904595                       0.0   
           -93                         0.904595                       0.0   
           -92                         0.904595                       0.0   

                           CNT_DRAWINGS_CURRENT  CNT_DRAWINGS_OTHER_CURRENT  \
SK_ID_CURR MONTHS_BALANCE                                                     
100013     -96                              0.0                         0.0   
           -95                              0.0                         0.0   
           -94                              0.0                         0.0   
           -93                              0.0                         0.0   
           -92                              0.0                         0.0   

                           ...  AMT_BALANCE_CREDIT_RATIO  \
SK_ID_CURR MONTHS_BALANCE  ...                             
100013     -96             ...                      10.0   
           -95             ...                      10.0   
           -94             ...                      10.0   
           -93             ...                      10.0   
           -92             ...                      10.0   

                           AMT_CREDIT_USE_RATIO  AMT_DRAWING_ATM_RATIO  \
SK_ID_CURR MONTHS_BALANCE                                                
100013     -96                         0.025281                   10.0   
           -95                         0.025281                   10.0   
           -94                         0.025281                   10.0   
           -93                         0.025281                   10.0   
           -92                         0.025281                   10.0   

                           AMT_DRAWINGS_OTHER_RATIO  AMT_DRAWINGS_POS_RATIO  \
SK_ID_CURR MONTHS_BALANCE                                                     
100013     -96                                  0.0                     0.0   
           -95                                  0.0                     0.0   
           -94                                  0.0                     0.0   
           -93                                  0.0    

Convert dataframe to 3D array (n_sample * n_time_step * n_features) for GRU network training.

In [7]:
train_wide = ccbl_trn.unstack(level='MONTHS_BALANCE', fill_value=-9)
test_wide = ccbl_test.unstack(level='MONTHS_BALANCE', fill_value=-9)

# 2. 获取维度信息，为重塑 (reshape) 做准备
n_train_samples = len(train_wide.index)
n_test_samples = len(test_wide.index)
n_features = len(ccbl_trn.columns)
n_timesteps = len(train_wide.columns) // n_features

# 3. 将2D宽数据重塑为3D数组，并交换维度以匹配GRU/LSTM的输入要求
#    目标维度: (样本数, 时间步长, 特征数)
train_x = train_wide.values.reshape(n_train_samples, n_features, n_timesteps)
train_x = np.swapaxes(train_x, 1, 2)

test_x = test_wide.values.reshape(n_test_samples, n_features, n_timesteps)
test_x = np.swapaxes(test_x, 1, 2)

# 4. train_y 的逻辑保持不变
#    假设 data_app 和 trn_id 已经定义
train_y = data_app['TARGET'].loc[data_app.SK_ID_CURR.isin(trn_id)]

# 5. 打印形状以验证
print("train_x shape:", train_x.shape)
print("test_x shape:", test_x.shape)
print("train_y shape:", train_y.shape)

train_x shape: (86905, 96, 32)
test_x shape: (16653, 96, 32)
train_y shape: (86905,)


Define GRU model. Use callback to evaluate auc metric.

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.regularizers import l2
from keras.optimizers import RMSprop, Adam

def build_model(time_step, n_features):
    model = Sequential()
    model.add(GRU(16, input_shape=(time_step, n_features))) #unit: #of neurons in each LSTM cell? input_shape=(time_step, n_features)
    model.add(Dense(1,activation='sigmoid'))
    return model

from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
import logging

class IntervalEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == (self.interval-1):
            y_pred = self.model.predict(self.X_val, verbose=0)[:,0]
            score = roc_auc_score(self.y_val, y_pred)
            print('roc score',score)

Training...

In [9]:
from tensorflow.keras.optimizers.legacy import Adam # for mac only
# Run a 5 fold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
oof_preds = np.zeros(train_x.shape[0])
sub_preds = np.zeros(test_x.shape[0])

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x, train_y)):
    trn_x, val_x = train_x[trn_idx], train_x[val_idx]
    trn_y, val_y = train_y.values[trn_idx], train_y.values[val_idx]
    ival = IntervalEvaluation(validation_data=(val_x, val_y), interval=5)
    
    model = build_model(trn_x.shape[1],trn_x.shape[2])
    model.compile(loss='binary_crossentropy', optimizer=Adam(decay=0.001))
    model.fit(trn_x, trn_y,
              validation_data= [val_x, val_y],
              epochs=20, batch_size=3000, 
              class_weight = {0:1,1:10},
              callbacks=[ival], verbose=5)
    
    oof_preds[val_idx] = model.predict(val_x)[:,0]
    sub_preds += model.predict(test_x)[:,0] / folds.n_splits
    
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))
  
    del model, trn_x, trn_y, val_x, val_y
    gc.collect()

2025-10-15 15:21:57.710859: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-10-15 15:21:57.711037: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-10-15 15:21:57.711053: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-10-15 15:21:57.711187: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-10-15 15:21:57.711206: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/20


2025-10-15 15:22:00.919963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:22:01.105721: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:22:01.296558: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:22:02.908868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:22:03.040603: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20


2025-10-15 15:22:10.452039: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:22:10.500936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

roc score 0.6153615244268923
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
roc score 0.6171914000257
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
roc score 0.6206912991567051
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
roc score 0.607146741772614
521/521 [==============================] - 3s 5ms/step
Fold  4 AUC : 0.607147
Epoch 1/20


2025-10-15 15:25:33.662852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:25:33.841300: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:25:34.013992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:25:36.093417: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:25:36.192576: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20


2025-10-15 15:25:43.204252: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:25:43.243852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


roc score 0.6209509751603097
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
roc score 0.6342800206904757
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
roc score 0.6382642832879597
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
roc score 0.6416575865063453
521/521 [==============================] - 3s 6ms/step
Fold  5 AUC : 0.641658


Save model prediction to disk.

In [11]:
cc_score_train = pd.DataFrame({'cc_score':oof_preds}, index=trn_id)
cc_score_test = pd.DataFrame({'cc_score':sub_preds}, index=test_id)             
cc_score_train.to_csv('../output/cc_score_train.csv')
cc_score_test.to_csv('../output/cc_score_test.csv')